In [ ]:
from transformers import AutoImageProcessor, DetrForObjectDetection
import torch
from PIL import Image

image_processor = AutoImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")


/home/john/dev/ai-osint-tracker/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:2409: UserWarning: for conv1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/home/john/dev/ai-osint-tracker/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:2409: UserWarning: for bn1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/home/john/dev/ai-osint-tracker/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:2409: UserWarning: for bn1.bias: copying from a non-meta parameter in the checkpoint

In [ ]:
image = Image.open("tesla.png").convert("RGB")

inputs = image_processor(images=image, return_tensors="pt")
outputs = model(**inputs)

In [24]:
# convert outputs (bounding boxes and class logits) to Pascal VOC format (xmin, ymin, xmax, ymax)
target_sizes = torch.tensor([image.size[::-1]])
results = image_processor.post_process_object_detection(outputs, threshold=0.9, target_sizes=target_sizes)[
    0
]
print(results["scores"])
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
        f"Detected {model.config.id2label[label.item()]} with confidence "
        f"{round(score.item(), 3)} at location {box}"
    )

tensor([0.9989, 0.9586, 0.9733], grad_fn=<IndexBackward0>)
Detected car with confidence 0.999 at location [9.0, 13.59, 295.71, 155.76]
Detected truck with confidence 0.959 at location [218.14, 2.61, 297.04, 44.0]
Detected surfboard with confidence 0.973 at location [165.59, 72.33, 218.71, 94.59]
